In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import descarteslabs as dl
import ee
import geopandas as gpd
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shapely
import shapely.geometry

from scripts import dl_utils

In [ ]:
def get_country_boundary(country_name):
    ee.Initialize()
    country = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(ee.Filter.eq('country_na', country_name));
    country_bounds = gpd.GeoDataFrame.from_features(country.getInfo()['features'], crs='WGS84')
    country_bounds.to_file(f'../data/boundaries/{country_name.lower()}.geojson')
    return country_bounds

def get_country_roi(country_name, pop_threshold=0.25, save=True, plot=True):
    ee.Initialize()

    dataset = ee.ImageCollection("WorldPop/GP/100m/pop");

    country = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(ee.Filter.eq('country_na', country_name));
    country_bounds = gpd.GeoDataFrame.from_features(country.getInfo()['features'], crs='WGS84')

    data = ee.Image(dataset.select('population').reduce(ee.Reducer.median())).clipToCollection(country);
    zones = data.gt(pop_threshold);
    zones = zones.updateMask(zones.neq(0));

    vectors = zones.addBands(data).reduceToVectors(
        geometry = country,
        crs = data.projection(),
        scale = 2000,
        geometryType = 'polygon',
        eightConnected = True,
        labelProperty = 'zone',
        reducer = ee.Reducer.mean())

    url = vectors.getDownloadURL()
    df = pd.read_csv(url)
    geoms = [shapely.geometry.Polygon(json.loads(g)['coordinates'][0]) for g in df['.geo']]
    polygons = gpd.GeoDataFrame(geometry=geoms, crs='EPSG:4326')

    area = polygons.to_crs('epsg:3857')['geometry'].area.sum() / 10 ** 6
    country_area = (country_bounds.to_crs('epsg:3857')['geometry'].area / 10 ** 6).values[0]
    print(f"{area / country_area:.3%}")

    if plot:
        fig, ax = plt.subplots(figsize=(8,5), dpi=150)
        country_bounds.plot(ax=ax)
        polygons.plot(ax=ax, color='r')
        plt.xticks([])
        plt.yticks([])
        plt.title(f"{country_name}, Pop Threshold {pop_threshold}")
        plt.show()
    if save:
        polygons.to_file(f'../data/boundaries/{country_name.lower()}_pop_{pop_threshold}.geojson', driver='GeoJSON')
    return polygons

In [ ]:
# Country name must be capitalized
country_name = 'Philippines'

In [ ]:
tilesize = 900
pad = 20
resolution = 10.0
boundary = get_country_boundary(country_name).dissolve()
tiles = dl.scenes.DLTile.from_shape(boundary, resolution, tilesize, pad)

In [ ]:
dlkey_features = list()
for tile in tiles:
    dlkey_feature = dict()
    dlkey_feature['geometry'] = shapely.geometry.mapping(tile.geometry)
    dlkey_feature['properties'] = dict()
    dlkey_feature['properties']['key'] = tile.key
    dlkey_features.append(dlkey_feature)

In [ ]:
dlkey_features = gpd.GeoDataFrame.from_features(dlkey_features)
dlkey_index = dlkey_features['geometry'].sindex

In [ ]:
pop_threshold = 0.5
polygons = get_country_roi(country_name, pop_threshold, save=False)

In [ ]:
overlap = []
for candidate in polygons['geometry']:
    indices = dlkey_index.query(candidate, predicate='intersects')
    overlap += list(indices)
   
union = dlkey_features.iloc[np.unique(overlap)]
# compute total area of union
union_area = (union.set_crs('epsg:4326').to_crs('epsg:3857')['geometry'].area / 10 ** 6).sum()
print(f"Total area of DLTiles (km^2): {union_area:,.0f}")
fig, ax = plt.subplots(figsize=(8,5), dpi=150)
union.plot(ax=ax)
plt.axis('equal')
plt.title(f"{len(union):,} tiles ({len(union) / len(dlkey_features):.1%}) in {country_name} are populated at a threshold of {pop_threshold}")
plt.xticks([])
plt.yticks([])
print(f"Approx DL Time: {len(union) / 500:.2f} hours")
print(f"Number of DL Tiles to process: {len(union)}")
plt.show()

dl_utils.write_dlkeys(union['key'], f'../data/boundaries/dlkeys/{country_name.lower()}_pop_{pop_threshold}_dlkeys.txt')
dl_union = union[['geometry', 'key']]
dl_union.to_file(f'../data/boundaries/{country_name.lower()}_pop_{pop_threshold}_dltiles.geojson', driver='GeoJSON')